Tensorflow 101
---- 

Este notebook é um material de apoio ao Webinar sobre Deep Learning e Tensorflow que foi ao ar no dia 31/08

O objetivo desse Notebook é demonstrar os aspectos básicos acerca do Tensorflow sem esgotar suas funcionalidades e capacidades.

Antes de começar, certifique-se de que está com todas as dependências instaladas, executando:

    pip install -r requirements.txt

As células que aparecem aqui são as mesma demonstradas no webinar

Primeiro importamos o Tensorflow e as outras bibliotecas

In [1]:
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
from datetime import datetime

Agora vamos criar algumas variáveis e performar uma simples conta matemática. Vale lembrar que o Tensorflow primeiro cria um grafo computacional com todas as variáveis, as inicializa e depois as computa.

Nessa primeira etapa vamos apenas criar um grafo com as seguintes variáveis

In [4]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

Criamos, em seguida, um inicializador de variáveis

In [5]:
init = tf.global_variables_initializer() 

Agora temos que iniciar uma seção do Tensorflow para processar nosso resultado|

In [ ]:
with tf.Session() as sess:
    init.run()
    result = f.eval()
print(result)

42!

Acabamos de calcular a resposta da vida, universo e tudo mais utilizando o Tensorflow!!!

Pode não parecer algo realmente importante, mas na verdade o Tensorflow fez esse cálculo utilizando um nível muito mais baixo do que o python consegue utilizar e isso é o que faz dele um framework especial.

Agora sabemos que o Tensorflow está funcionando, vamos fazer uma pequena rede neural para identificar dígitos escritos à mão.

Machine Learning com Tensorflow
---

Para esse modelo vamos usar imagens de dígitos escritos a mão, cada dígito foi convertido para uma imagem de 28X28 com pixels de 0 até 255. Primeiro extraímos os dados

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data')

Podemos vizualizar a imagem

In [ ]:
batch_xs, batch_ys = mnist.test.next_batch(1)
two_d = (np.reshape(batch_xs, (28, 28)) * 255).astype(np.uint8)
plt.imshow(two_d, 'gray',interpolation='nearest')
plt.show()

print "Número da imagem: " + str(batch_ys[0])


Como o Tensorflow primeiro constrói um grafo para depois calcular todas as variáveis, primeiro temos que construir a estrutura da nossa rede neural.

Como estamos lidando com imagens, podemos pensar que cada pixel é um input que vai passar por um neurônio, entretanto se essa estivéssemos construíndo uma rede convolucional essa ideia seria um pouco diferente.

A rede que criaremos a seguir é uma Dense Neural Netowork. Nesta rede existe um neurônio para cada input e todos os neurônios de uma camada estão ligados aos neurônios da próxima camada.

Vamos primeiro definir qual será a função de construção das camadas

In [4]:
def dense_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        initializer = tf.contrib.layers.xavier_initializer()
        init=tf.truncated_normal((n_inputs,n_neurons), stddev=2/np.sqrt(n_inputs))
        w = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name = "biases")
        z = tf.matmul(X, w) + b
        if activation:
            return tf.nn.relu(z)
        else:
            return z

O código acima está apenas construindo a estrutura de uma camada densa de neurônios.
A função recebe os inputs X, a quantidade de neurônios presentes na camada, o nome da camada e se tem ou não um ativação ao final de cada neurônio

Em seguida, define o escopo ("nome da camada") no grafo do Tensorflow com o nome escolhido.
O número de inputs é deduzido a partir dos próprios inputs.
Aos pesos (weights) são atribuídos valores aleatórios normalmente.
Aos viés são atribuídos o valor 0.

Vale lembrar que a função de um neurônio é dada por Y = f((w*x)+b). Desta equação já definimos o 'w' e o 'b', agora temos que definir a função de ativação.

Nesse caso a função de ativação será a função ReLU (rectified linear unit), essa função é dada por z = max(0,x). Ou seja retorna zero sempre que x<0 ou x se x>0.

Para definir essa função utilizamos o método nativo do Tensorflow tf.nn.relu()

Construindo a rede neural
----

Agora que definimos nossa função de camadas, vamos construir nossa rede neural.

Primeiro temos que dizer para o Tensorflow qual é o tipo do dado de entrada e saída. Para isso, utilizamos o método placeholder e passamos o tipo do dado, a forma do dado e o nome do placeholder. Note que os dados de entrada têm uma forma (None, 28*28), no qual o primeiro valor se refere ao tamanho do batch de treinamento/predição e o segundo valor é o tamanho de cada imagem (nesse caso, nossas imagens têm tamanho 28X28). Vale lembrar que as imagens serão transformadas de uma matriz 28X28 para um array de tamanho 784.

Em seguida criamos a rede neural em si. Essa rede simples performa muito bem nesse dataset, e é composta de 3 camadas com com a seguinte composição:
    - layer 1: 300 neurônios
    - layer 2: 100 neurônios
    - layer 3: 10 neurônios - camada de saída
    
A saída desta rede neural é chamada de logits e é um array de 10 valores, cada um representando um neurônio. Isso por que em vez da rede responder um único dígito de 0-9, irá retornar um array com a ativação de cada um dos neurônios. Ou seja, podemos dizer que cada neurônio está responsável para identificar um único dígito. Posteriormente, iremos utilizar uma função chamada de softmax para normalizar as saídas em valores entre 0. e 1. que representam a própria probabilidade de confiança de cada neurônio.

In [5]:
# Define os placeholders de entrada e saída
X=tf.placeholder(tf.float32,(None,28*28),name="X")
y=tf.placeholder(tf.int64,shape=(None),name="y")

# Cria a rede utilizando a função definida anteriormente
with tf.name_scope("dnn"):
    layer1= dense_layer(X, 300, "hidden1", activation=True)
    layer2= dense_layer(layer1, 100, "hidden2", activation=True)
    logits= dense_layer(layer2, 10, "logits")

Para o processo de treinamento poder acontecer precisamos definir um parâmetro de erro/perda a ser minimizado pelo gradient descent. Nesse caso, vamos utilizar uma função de cross-entropia

In [6]:
# Define a os parâmetros de perda
with tf.name_scope("loss"):
    xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss=tf.reduce_mean(xentropy,name="loss")    

Agora parametrizamos o valor da taxa de aprendizado, que dita a quantidade que o gradiente irá variar em cada interação. Uma taxa muito grande pode fazer com que a rede passe do ponto e não consiga convergir para o ponto ótimo. Por outro lado, uma taxa muito pequena vai impedir que o aprendizado aconteça

O Tensorflow toma conta de fazer todo o Back Propagation automaticamente! Para isso precisamos criar um operador para o otimizador e passar qual valor deve ser minimizado. É por meio desse operador que o Tensorflow irá atualizar todos os pesos e viés da rede.

In [7]:
# Cria um otimizador e parametriza um operador para esse otimizador
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer=tf.train.GradientDescentOptimizer(learning_rate)
    training_op=optimizer.minimize(loss)

A última coisa antes de começar a treinar a rede é definir algumas métricas para acompanhar o processo de treinamento. A métrica que vamos definir a seguir é a acuracidade: o % de acerto de um batch

In [ ]:
# Cria as variáveis de avaliação
with tf.name_scope("eval"):
    correct= tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    conf = tf.reduce_max(tf.nn.softmax(logits, axis=1), axis=1)

    tf.summary.scalar("Accuracy",accuracy)

init = tf.global_variables_initializer()
saver = tf.train.Saver()
merged = tf.summary.merge_all()

Treinando o Modelo
---

Por fim, vamos treinar o modelo. Antes vamos definir o duas coisas:
    - Época: uma iteração pelo dataset de treino
    - Batch: um subset do dataset de treino
    
O código abaixo seleciona um batch por vez e o fornece para o operador de treino com:

    for iteration in range(mnist.train.num_examples//batch_size):
        X_batch, y_batch=mnist.train.next_batch(batch_size)
        sess.run(training_op,feed_dict={X:X_batch,y:y_batch})

O operador de treino tem por objetivo realizar o gradient descent com o intuito de minimizar o erro final do modelo. Isso acontece por meio do ajuste de todos os pesos e viés da rede, na realidade a matemática por trás disso é um tanto complexa e não cabe explicá-la aqui.

Toda a vez que o dataset de treino é lido por inteiro, o dataset de test é utilizado para avaliar o modelo.

Com apenas 40 épocas esse modelo consegue atingir um resultado de 97% de acuracidade!

In [ ]:
n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    # Cria os arquivos do tensorboard
    now = datetime.utcnow()
    train_writer = tf.summary.FileWriter('logdir/train-{}'.format(now),sess.graph)
    test_writer = tf.summary.FileWriter('logdir/test-{}'.format(now))
    
    # Inicia as variáveis do tensorflow
    init.run()
    
    for epoch in range(n_epochs):
    
        # Itera n baches do dataset
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch, y_batch=mnist.train.next_batch(batch_size)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
            
        # Adiciona as métricas para o Tensorboar
        train_writer.add_summary(merged.eval(feed_dict={X:X_batch,y:y_batch}),epoch)
        test_writer.add_summary(merged.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels}),epoch)

        # Calcula métricas de acurácia para acompanhar o treinamento
        acc_train=accuracy.eval(feed_dict={X:X_batch,y:y_batch})
        acc_test=accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
        print(epoch,"Train accuracy:",acc_train,"Test accuracy:",acc_test)
        
        # Salva os checkpoits por cada Época
        save_path=saver.save(sess,"./logdir/my_model_final.ckpt")

Utilizando o Modelo
---

Por fim, utilizamos o mesmo modelo para classificar uma imagem do dataset de treino.

Nesse caso o output do modelo são logits, onde o index do maior argumento representa o número classificado. Se uma função softmax for utilizada nesses logits tem-se a probabilidade da classificação 

In [ ]:
batch_xs, batch_ys = mnist.test.next_batch(1)

with tf.Session() as sess:
    saver.restore(sess, "./logdir/my_model_final.ckpt")
    X_new_scaled=batch_xs
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred=np.argmax(Z, axis=1)
    confidence = sess.run(tf.nn.softmax(Z, axis=1))

print "Prediction: {} | Confidence: {}%".format(y_pred[0],confidence[0][y_pred][0] * 100)
    
two_d = (np.reshape(batch_xs, (28, 28)) * 255).astype(np.uint8)
plt.imshow(two_d, 'gray',interpolation='nearest')
plt.show()
